In [1]:
import math
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from numpy import sin, cos, arccos, pi, round

from sklearn.impute import KNNImputer

from sklearn import neighbors
from sklearn import linear_model
from sklearn import svm
from sklearn import tree
from sklearn.metrics import mean_squared_error, mean_absolute_error

### 1° Leitura e tratamento inicial dos dados

Lendo a CSV e removendo todos os dados com valores de Tp_est iguais a zero

Resultados:
- Base de dados raw para treinamento (*df_raw_train*)
- Base de dados raw para validação (*df_raw_validation*)
- Base de dados raw para teste (*df_raw_test*)

In [2]:
df_raw_data = pd.read_csv('/content/drive/MyDrive/BCC - UFPR/Semestres /9 - 2023-2/Aprendizado de Maquina/Lab2/Dados_Radar_Estacao_Completo_2018_2022.csv')
df_raw_data.drop(df_raw_data[df_raw_data['Tp_est'] == 0.0].index, inplace=True);

Remoção inicial de colunas da base geral (2018 a 2022). A colunas removidas foram:
- Unnamed: 0 pois é a coluna de ids
- latitude e longitude pois possuem a mesma informação que as colunas lat e lon;
- distancia: Não agrega valor ao modelo. (*)

In [3]:
df_raw_data.drop(['Unnamed: 0', 'latitude', 'longitude', 'distancia'], axis=1, inplace=True)

Separando o raw data entre treinamento e teste. Anos de 2018 a 2021 para treinamento e 2022 para teste.


In [4]:
raw_train_test_group = df_raw_data.groupby(df_raw_data['time'].str.contains('2022'))

df_raw_train = raw_train_test_group.get_group(False).copy()
df_raw_test = raw_train_test_group.get_group(True).copy()

Separando os dados de treinamento, selecionando uma porção dos dados para validação

In [5]:
# dates = ['2018-01', '2018-02']
dates = ['2018-01']

raw_train_validation_group = df_raw_train.groupby(df_raw_train['time'].str.contains('|'.join(dates)))
df_raw_train = raw_train_validation_group.get_group(False).copy()
df_raw_validation = raw_train_validation_group.get_group(True).copy()


Remoção das colunas elevation e sweep pois não possuiam valor agregado na base

Para verificar que as colunas elevation e sweep não possuiam valor agregado, foi utilizado a função describe do pandas. Com esta função, foi possível verificar que ambas possuiam média, minimo e maximo identicos, além de um desvio padrão igual a 0, ou seja, todas as linhas possuiam o mesmo valor.

In [6]:
print(df_raw_train.describe()['elevation'], end="\n\n")
print(df_raw_train.describe()['sweep'])

df_raw_train.drop(['elevation', 'sweep'], axis=1, inplace=True)
df_raw_validation.drop(['elevation', 'sweep'], axis=1, inplace=True)
df_raw_test.drop(['elevation', 'sweep'], axis=1, inplace=True)

count    85441.0
mean         0.5
std          0.0
min          0.5
25%          0.5
50%          0.5
75%          0.5
max          0.5
Name: elevation, dtype: float64

count    85441.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
Name: sweep, dtype: float64


### 3° Imputação de dados na base de treinamento

Resultados:
- Base de treinamento com todas as linhas com valores preenchidos (*df_train*)

In [7]:
df_train = df_raw_train.copy()

Realizando o input na base de dados de treinamento

In [18]:
teste = df_train.drop(['time', 'Est'], axis=1)
teste_na = teste.dropna()

teste_list = np.array(teste.values.tolist())
na_list = np.array(teste_na.values.tolist())

imputer = KNNImputer(n_neighbors=2)
teste_list = imputer.fit_transform(teste_list, na_list)

In [19]:
teste_list[1]

array([ 1.91000000e+02,  7.27500000e+04,  1.05000000e-01, -2.45000000e+00,
       -1.18500000e+00,  6.80000000e-01, -1.75000000e-01,  5.00000000e-01,
        4.20000000e-01, -1.38794500e+04, -7.14035800e+04,  9.46240000e+02,
       -2.55120851e+01, -5.36675815e+01,  1.06623927e+03,  2.00000000e-01])

In [20]:
teste2 = df_train.drop(['time', 'Est'], axis=1)
teste_na2 = teste.dropna()

teste_list2 = np.array(teste2.values.tolist())
na_list2 = np.array(teste_na2.values.tolist())

imputer = KNNImputer(n_neighbors=2)
teste_list2 = imputer.fit_transform(teste_list2)

In [21]:
teste_list2[1]

array([ 1.91000000e+02,  7.27500000e+04,  1.05000000e-01, -2.45000000e+00,
       -1.18500000e+00,  6.80000000e-01, -1.75000000e-01,  5.00000000e-01,
        4.20000000e-01, -1.38794500e+04, -7.14035800e+04,  9.46240000e+02,
       -2.55120851e+01, -5.36675815e+01,  1.06623927e+03,  2.00000000e-01])

### Remoção de outliers

In [ ]:
data = { '0 - 2.4': 0, '2.5 - 7.4': 0, '7.5 - 12.4': 0, '12.5 - 17.4': 0, '17.5 - 22.4': 0, '22.5 - 27.4': 0, '27.5 - 32.4': 0,  '32.4 - 37.5': 0}
for row in df_train.iterrows():
    if float(row[1]['Tp_Est']) < 2.4:
        data['0 - 2.4'] += 1
    elif float(row[1]['Tp_Est']) < 7.4:
        data['2.5 - 7.4'] += 1
    elif float(row[1]['Tp_Est']) < 12.4:
        data['7.5 - 12.4'] += 1
    elif float(row[1]['Tp_Est']) < 17.4:
        data['12.5 - 17.4'] += 1
    elif float(row[1]['Tp_Est']) < 22.4:
        data['17.5 - 22.4'] += 1
    elif float(row[1]['Tp_Est']) < 27.4:
        data['22.5 - 27.4'] += 1
    elif float(row[1]['Tp_Est']) < 32.4:
        data['27.5 - 32.4'] += 1
    else
        data['32.4 - 37.5'] += 1

data

### 4° Preparação da base de validação
Realizar a imputaçao de dados na base de validação

Resultados:
- Base de validação com todas as linhas com valores preenchidos (*df_validation*)

In [ ]:
df_validation = df_raw_validation.copy()

Realizando o input na base de dados de validação

In [ ]:
def get_fields_to_input(row, columns):
    fields = []
    for col in columns:
        if str(row[col]) == 'nan':
            fields.append(col)
    return fields

def get_data(timestamp, input_values):
    data = None
    min_distance = math.inf
    for value in input_values:
        distance = abs(timestamp - value["time"])
        if distance < min_distance:
            min_distance = distance
            data = value

    return data

columns = list(df_validation.head())
for row in df_validation.iterrows():
    timestamp = datetime.strptime(row[1]["time"], '%Y-%m-%d %H:%M:%S').timestamp()

    fields_to_input = get_fields_to_input(row[1], columns)
    if len(fields_to_input) > 0:
        input_data = get_data(timestamp, input_values)
        for field in fields_to_input:
            if str(row[1][field]) == 'nan':
                df_validation.loc[row[0], field] = input_data[col]

### Validaçao dos modelos de regressão

In [ ]:
df_train_for_validation = df_train.drop(['x', 'y', 'z', 'lat', 'lon', 'alt', 'time'], axis=1)
df_validation_for_train = df_validation.drop(['x', 'y', 'z', 'lat', 'lon', 'alt', 'time', 'Est'], axis=1)

In [ ]:
df_train_for_validation = df_train.drop(['time'], axis=1)
df_validation_for_train = df_validation.drop(['time','Est'], axis=1)

In [ ]:
def get_model(model):
    match model:
        case 'knn':
            return neighbors.KNeighborsRegressor(n_neighbors=100)
        case 'svr':
            return svm.SVR()
        case 'linear_regression':
            return linear_model.LinearRegression()
        case 'tree_regression':
            return tree.DecisionTreeRegressor()


models = ['knn', 'svr', 'linear_regression', 'tree_regression']
est_preds = {}

validate_list = np.array(df_validation_for_train.values.tolist())
x_validate = validate_list[:, :-1]
y_validate = validate_list[:, -1]

grouped_by_est = df_train_for_validation.groupby(['Est'])
for est in grouped_by_est.groups.keys():
    group = grouped_by_est.get_group(est).copy()
    group = group.drop(['Est'], axis=1)
    train_list = np.array(group.values.tolist())
    x_train = train_list[:, :-1]
    y_train = train_list[:, -1]

    preds = {}
    for model_type in models:
        model = get_model(model_type)
        model.fit(x_train, y_train)

        pred = model.predict(x_validate)
        preds[model_type] = pred

    est_preds[est] = preds

In [ ]:
errors = {}
for est in est_preds:
    est_errors = {}
    for model in est_preds[est]:
        mse = mean_squared_error(y_validate, est_preds[est][model])
        mae = mean_absolute_error(y_validate, est_preds[est][model])
        est_errors[model] = { 'mse': mse, 'mae': mae }
    errors[est] = est_errors

In [ ]:
df_comparison = pd.DataFrame(columns=['Estação', 'Melhor Modelo MSE', 'MSE', 'Melhor Modelo MAE', 'MAE'])

for est in errors:
    min_mse = None
    min_mse_value = math.inf
    min_mae = None
    min_mae_value = math.inf

    for model in errors[est]:
        if errors[est][model]['mse'] < min_mse_value:
            min_mse = model
            min_mse_value = errors[est][model]['mse']
        if errors[est][model]['mae'] < min_mae_value:
            min_mae = model
            min_mae_value = errors[est][model]['mae']
    new_row = [est, min_mse, min_mse_value, min_mae, min_mae_value]
    df_comparison.loc[len(df_comparison)] = new_row

df_comparison

,Estação,Melhor Modelo MSE,MSE,Melhor Modelo MAE,MAE
0,Aguas_do_Vere,knn,6.669371,svr,1.066098
1,Altonia,knn,6.584339,svr,1.058196
2,Assis_Chateaubriand,knn,6.671869,svr,1.058294
3,Baixo_Iguacu,knn,6.772601,svr,1.058378
4,Bela_Vista_Jusante,knn,6.573816,svr,1.058450
5,Boa_Vista_da_Aparecida,knn,6.702941,svr,1.058236
6,Campo_Mourao,knn,6.574060,svr,1.058340
7,Cascavel,knn,6.469714,svr,1.057651
8,Coronel_Domingos_Soares,knn,6.544976,svr,1.066966
9,Derivacao_do_Rio_Jordao,knn,6.921091,svr,1.058348
